# AKI Seminar2 Demo
## Finetuning of LLaMA

by Syon Kadkade


Table of contents

> [Install packages](#install)   
> [Import libaries](#imports)   
> [Lorem Ipsum]()


In [1]:
!nvidia-smi

Thu Jan 18 13:35:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

--------------
<a id="install"></a>
### Install packages[Emoji]

**Description**:   
lorem ipsum

In [2]:
!pip install accelerate --quiet
!pip install bitsandbytes --quiet
!pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00


In [3]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 2.3 MB/s eta 0:00:00


---------------
<a id="imports"></a>
### Import libaries [Emoji]
**Description**:   
Load all necessary libaries.

In [4]:
import numpy as np
import pandas as pd
import torch
import transformers
from accelerate import Accelerator
from transformers import DataCollatorWithPadding, LlamaForCausalLM, LlamaTokenizer

In [5]:
import warnings
warnings.filterwarnings('ignore')

-----------------
### Load LLaMA-7B-Model[Emoji]

**Description**:  
lorem ipsum dolor sit amet

**Resources**:
- [Tutorial](#https://www.youtube.com/watch?v=t68IV5t5UOA)
- [Hugging Face: Transformer Tutorial](#https://huggingface.co/learn/nlp-course/chapter2/2?fw=pt)
- [Hugging Face: LLaMA-7B-Model](https://huggingface.co/docs/transformers/main/model_doc/llama)
- [Hugging Face: LLaMA weights](https://huggingface.co/luodian/llama-7b-hf)
- [Hugging Face: 7B Weights](#https://huggingface.co/huggyllama/llama-7b)

**Note**: I use a model that has the weights in it and we introduce these into the actual LLaMA model. Normally you have to request the weights from Meta AI by filling out a form. I have filled it out several times but there is no response from them.

In [6]:
#MODEL_NAME = 'huggyllama/llama-7b'
MODEL_NAME = "Enoch/llama-7b-hf"

In [7]:
tokenizer = LlamaTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token_id = (0)
tokenizer.padding_side = "left"

tokenizer_config.json:   0%|          | 0.00/218 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
model = LlamaForCausalLM.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/511 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/25.5k [00:00<?, ?B/s]

pytorch_model-00001-of-00033.bin:   0%|          | 0.00/405M [00:00<?, ?B/s]

pytorch_model-00002-of-00033.bin:   0%|          | 0.00/405M [00:00<?, ?B/s]

pytorch_model-00003-of-00033.bin:   0%|          | 0.00/405M [00:00<?, ?B/s]

...

pytorch_model-00031-of-00033.bin:   0%|          | 0.00/405M [00:00<?, ?B/s]

pytorch_model-00032-of-00033.bin:   0%|          | 0.00/405M [00:00<?, ?B/s]

pytorch_model-00033-of-00033.bin:   0%|          | 0.00/524M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/151 [00:00<?, ?B/s]

--------------
### Example Usage

In [19]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
#inputs = tokenizer(raw_inputs, padding='max_length', truncation=True, return_token_type_ids=True)
inputs = tokenizer(raw_inputs, truncation=True, return_tensors="np") #Here we don't use padding because it is ineffecient. Better use after batching.
print(inputs)

{'input_ids': array([array([    1,   306, 29915,   345,  1063, 10534,   363,   263,   379,
                688,  3460, 23360,  3236,   590,  3353,  2834, 29889])      ,
       array([    1,   306, 26277,   445,   577,  1568, 29991])],
      dtype=object), 'attention_mask': array([array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
       array([1, 1, 1, 1, 1, 1, 1])], dtype=object)}


In [12]:
inputs["input_ids"]

[[1,
  306,
  29915,
  345,
  1063,
  10534,
  363,
  263,
  379,
  688,
  3460,
  23360,
  3236,
  590,
  3353,
  2834,
  29889],
 [1, 306, 26277, 445, 577, 1568, 29991]]

In [13]:
decoded_string = tokenizer.decode(inputs["input_ids"][0])
print(decoded_string)

<s>I've been waiting for a HuggingFace course my whole life.


--------------
### Create own Dataset

In [ ]:
from json import loads, dumps
df = pd.DataFrame(
     [["a", "b"], ["c", "d"]],
     index=["row 1", "row 2"],
     columns=["col 1", "col 2"],
 )

In [ ]:
from datasets import load_dataset
data_files = {"train": "SQuAD_it-train.json", "test": "SQuAD_it-test.json"}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")
squad_it_dataset
issues_dataset = load_dataset("json", data_files="datasets-issues.jsonl", split="train")
issues_dataset

----------------
### Load Alpaca dataset

In [22]:
from datasets import load_dataset

dataset = load_dataset("vicgalle/alpaca-gpt4")

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

-------------
### Data Preprocessing

In [11]:
def tokenize_function(data):
  """
  A function to tokenize given sentences.

  ...
  """
  return tokenizer(data["sentence1"], data["sentence2"], padding='max_length', truncation=True, )

In [ ]:
#Use Dataset.map() function to tokenize entire dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

In [ ]:


#We can use the Collator function from huggingface to apply dynamic padding to our data.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

----------------
### Prepare Trainer for finetuning
Do it via Trainer API or with own training pipeline

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

----------------
### Finetune Model

In [ ]:
trainer.train()

----------------
### Save Model and Tokenizer

In [21]:
outputs = model(inputs)

AttributeError: 

In [ ]:
tokenizer.save_pretrained("./content/")

In [ ]:
model.save_pretrained("./content/")